In [1]:
#| default_exp gkm_to_kmers

In [28]:
#| export
import pandas as pd, numpy as np
from collections import defaultdict
from Bio import SeqIO

from modules.feat_imp import *

In [3]:
exp = pd.read_csv('sp1_ml_datasets/ls-gkm/gkmexp_pos_impscores.txt', sep='\t', header=None, names=['seq_id', 'total_score', 'raw_contrib'])
exp

,seq_id,total_score,raw_contrib
0,chr20:257446-257646,-0.194996,"0,-0.000319389,0,0;0,0,-0.000640015,0;0,-0.001..."
1,chr20:290327-290527,0.452142,"0.00029,0,0,0;0.000121902,0,0,0;-2.98632e-05,0..."
2,chr20:297242-297442,0.238586,"1.85699e-05,0,0,0;-3.96274e-05,0,0,0;3.87967e-..."
3,chr20:310576-310776,0.647026,"0,-7.69097e-05,0,0;0,0,8.63976e-05,0;0,-0.0004..."
4,chr20:324472-324672,0.366285,"0,0,-0.000366061,0;0,0,-0.000560686,0;0,0,0,-0..."
...,...,...,...
195,chrX:3814879-3815079,0.023696,"0,0,1.80487e-05,0;1.78979e-05,0,0,0;0,-0.00027..."
196,chrX:3817555-3817755,0.221365,"2.78203e-05,0,0,0;0,0,0,-0.000220862;0,0,0.000..."
197,chrX:3817777-3817977,-0.222220,"0,0.000378686,0,0;0,0.000657848,0,0;0,0,0,0.00..."
198,chrX:3872336-3872536,0.268282,"8.64888e-05,0,0,0;0,0,0,-2.25235e-05;0,0,0,-0...."


In [4]:
#| export
def fa_dict(path_to_fa):
    seqs = {}
    for record in SeqIO.parse(path_to_fa, 'fasta'):
        seqs[record.id] = str(record.seq).upper()
    return seqs

In [5]:
seqs = fa_dict('sp1_ml_datasets/ls-gkm/test_all_200bp.fa')

In [6]:
seqs['chr20:257446-257646']

'CATGTCCTCAACAAAGTTAAGCATGACTCCGTTAGAGATGGCTGTCCGAGAATTGTCAGGACTCAACGTCCTGGACACAGTAACTGCTGCTATTTACTAAGAAAAATCCTCATCTCTCAAGCACATAGACTCTCCCCTCTCCACCTAGCAGGGAGATATTACTCCCGCTTACAACTTCGTGCAGCTGAGGACCGCCCACA'

In [7]:
#| export
def parse_contrib(ref_base, pos_vec):
    mapping = {'A':0, 'C':1, 'G':2, 'T':3}
    base_contrib = list(map(float, pos_vec.split(',')))
    return base_contrib[mapping[ref_base]]

In [ ]:
# kmer_imp = defaultdict(float)
# kmer_counts = defaultdict(int)

# for i in range(len(exp)):
#     contribs = exp.loc[i,'raw_contrib']
#     pos_vec = contribs.split(';')
#     base_scores = [parse_contrib(v) for v in pos_vec]

#     fa_header = exp.loc[i,'seq_id']
#     seq = seqs[fa_header]

#     for i in range(len(seq) - k + 1):
#         kmer = seq[i:i+k]
#         score = sum(base_scores[i:i+k])
#         kmer_imp[kmer] += score
#         kmer_counts[kmer] += 1

In [8]:
#| export
def exp_to_kmer_imp(exp, seqs, k):
    kmer_imp = defaultdict(float)
    kmer_counts = defaultdict(int)

    for i in range(len(exp)):
        fa_header = exp.loc[i,'seq_id']
        seq = seqs[fa_header]
        contribs = exp.loc[i,'raw_contrib']
        pos_vec = contribs.split(';')
        base_scores = [parse_contrib(b,v) for b,v in list(zip(seq, pos_vec))]

        for i in range(len(seq) - k + 1):
            kmer = seq[i:i+k]
            score = sum(base_scores[i:i+k])
            kmer_imp[kmer] += score
            kmer_counts[kmer] += 1

    return kmer_imp, kmer_counts

In [9]:
kmer_imp, kmer_counts = exp_to_kmer_imp(exp, seqs, k=7)

In [10]:
len(kmer_imp)

13074

In [11]:
top_kmers = sorted(kmer_imp.items(), key=lambda x: abs(x[1]), reverse=True)
top_kmers[:20]

[('TTTTTTT', 1.2460853610000004),
 ('AAAAAAA', 1.2387053480000005),
 ('CAAAAAA', 0.335751349),
 ('AAAAAAG', 0.260321395),
 ('GAAAAAA', 0.2563059413),
 ('GCTCCCC', 0.24128265449999997),
 ('CCCCGCC', 0.230523687),
 ('AAAAAAC', 0.21706574069999995),
 ('GGGGCAG', 0.2164079969),
 ('CTTTTCT', 0.21501909869999997),
 ('GCGGCGG', 0.20314109430000002),
 ('TTTTTTA', 0.1963537847),
 ('AAGTGAG', 0.1959730935),
 ('ATTTTTT', 0.1940418647),
 ('TGAAAAA', 0.1927164251),
 ('AGGCCTC', 0.1788299489),
 ('AAAAAGG', 0.17813057000000002),
 ('TTTTTAT', 0.17762971),
 ('TGGGGGG', 0.17476472820000005),
 ('AGCCCAG', 0.172426984)]

In [12]:
norm_kmer_imp = {k:kmer_imp[k]/kmer_counts[k] for k in kmer_imp.keys()}

In [13]:
norm_top_kmers = sorted(norm_kmer_imp.items(), key=lambda x: x[1], reverse=True)
norm_top_kmers[:20]

[('CGCAAAA', 0.153058),
 ('TTTATCG', 0.0854356),
 ('TCTACGT', 0.0805933),
 ('CCCGGTG', 0.07870644),
 ('GGAGAAA', 0.07829457999999999),
 ('AGTGAAT', 0.0775288),
 ('CTTGGCT', 0.0770827),
 ('GCAAAAC', 0.075691518),
 ('TACGTGT', 0.0752115),
 ('CAGGTCG', 0.0722555),
 ('CCGGTGT', 0.0708969),
 ('GAAGTGA', 0.07011770375000001),
 ('TTGGCTA', 0.0680197),
 ('GCCGAAT', 0.0629512),
 ('CGGTGTG', 0.062696935),
 ('CTACGTG', 0.0602625),
 ('GAAGGAC', 0.059814),
 ('TTGAAAT', 0.05970400000000001),
 ('CACGGAC', 0.057403800000000005),
 ('GTTCGTC', 0.0569977)]

## Assessing motif learning

In [14]:
all_kmers = list(kmer_imp.keys())

In [15]:
pwm = load_jaspar_pwm('MA0079.1.jaspar').T
pwm

array([[0.25 , 0.125, 0.5  , 0.125],
       [0.   , 0.   , 1.   , 0.   ],
       [0.   , 0.   , 1.   , 0.   ],
       [0.   , 0.625, 0.25 , 0.125],
       [0.25 , 0.   , 0.5  , 0.25 ],
       [0.   , 0.125, 0.625, 0.25 ],
       [0.   , 0.   , 0.75 , 0.25 ],
       [0.125, 0.125, 0.75 , 0.   ],
       [0.25 , 0.   , 0.   , 0.75 ]])

In [16]:
kmer_scores = {k:best_pwm_score(k,pwm) for k in all_kmers}
len(kmer_scores)

13074

In [17]:
bg_scores = get_background_dist(all_kmers, 50, kmer_scores)
bg_scores.shape

(10000,)

In [18]:
top_kmer_scores = [kmer_scores[k[0]] for k in norm_top_kmers[:50]]
observed = np.mean(top_kmer_scores)
pval = (np.sum(bg_scores >= observed) + 1) / (10000 + 1)
print(f"Observed mean score: {observed:.3f}")
print(f"Null mean ± std: {bg_scores.mean():.3f} ± {bg_scores.std():.3f}")
print(f"p-value: {pval:.3e}")

Observed mean score: 2.712
Null mean ± std: 2.726 ± 0.093
p-value: 5.564e-01


In [30]:
def has_cg(k):
    return "CG" in k

In [33]:
def bg_cg_fraction(all_kmers, n_samp, n_iter=10000):
    rng = np.random.default_rng()
    null_stats = []
    for _ in range(n_iter):
        sampled = rng.choice(all_kmers, size=n_samp, replace=False)
        stat = np.mean([has_cg(k) for k in sampled])
        null_stats.append(stat)
    return np.array(null_stats)

In [34]:
fraction_top = np.mean([has_cg(k) for k,_ in norm_top_kmers[:50]])
fraction_nulls = bg_cg_fraction(all_kmers, 50)

In [36]:
(np.sum(fraction_nulls >= fraction_top) + 1) / (10000 + 1)

0.00019998000199980003

In [26]:
import nbdev.export as nb
nb.nb_export('06_lsgkm_imp_sp1.ipynb', './modules')